In [30]:
import numpy as np
from random import random

In [31]:
# save activations and derivatives
# implement backpropagation
# implement gradient decent
# implement train
# train our net woth some dummy dataset
# make some prediction

class MLP:

  def __init__(self, num_inputs=3, num_hidden=[3, 3], num_outputs=2):
    """
    MLP constructor
    Args:
      num_inputs (int): numero de entradas
      num_hidden (list): numero de neuronas ocultas
      num_outputs (int): numero de salidas
    """
    self.num_inputs = num_inputs
    self.num_hidden = num_hidden
    self.num_outputs = num_outputs

    # crear representacion generica de las capas
    layers = [num_inputs] + num_hidden + [num_outputs]

    weights = []
    for i in range(len(layers) - 1):
      w = np.random.rand(layers[i], layers[i + 1])
      weights.append(w)
    self.weights = weights

    derivatives = []
    for i in range(len(layers)-1):  #Si tenemos tres capas tenemos dos matrices W
      a = np.zeros((layers[i], layers[i+1]))
      derivatives.append(a)
    self.derivatives = derivatives

    # crear representacion de las activaciones de las capas
    activations = []
    for i in range(len(layers)):
      a = np.zeros(layers[i])
      activations.append(a)
    self.activations = activations

  def forward_propagate(self, inputs):
    """
    Calcula la propagacion forward a traves de la red basado en las entradas
    Args:
      inputs (ndarray): entradas a la red
    Returns:
      activations (ndarray): salidas de la red
    """
    activations = inputs

    # la activacion de la primera capa
    self.activations[0] = activations

    # itera a traves de las capa de la red
    for i, w in enumerate(self.weights):
      # Calcula las entradas netas
      net_inputs = np.dot(activations, w)

      # Calcula la activacion
      activations = self._sigmoid(net_inputs)
      self.activations[i+1]= activations #guardamos la activacion

    # a_3 = s(h_3)
    # h_3 = a_2 * W_2 ---> i = 2

    return activations

  def back_propagate(self,error, verbose=False):
    """Backpropogates an error signal.
    Args:
        error (ndarray): The error to backprop.
    Returns:
        error (ndarray): The final error of the input
    """

    #Este metodo de backpropagation funciona para infita cantidad de capas

    # dE/dW_i = (y - a_[i+1]) * s'(h_[i+1]) * a_i ---> error * s' * ai
    # s'(h_[i+1]) = s(h_[i+1]) * (1 - s(h_[i+1]))
    # s(h_[i+1]) = a_[i+1]

    # dE/dw_[i-1] = (y - a_[i+1]) * s'(h_[i+1]) * W_i * s'(h_i) * a_[i-1]

    # recorremos desde la ultima capa hasta la primera (al reves)
    for i in reversed(range(len(self.derivatives))):
      activations  = self.activations [i+1]

      delta = error * self._sigmoid_derivative(activations ) #ndarray([0.1, 0.2]) --> ndarray([[0.1, 0.2]])
      delta_reshaped = delta.reshape(delta.shape[0], -1).T

      current_activations = self.activations[i] #ndarray([0.1, 0.2]) --> ndarray([0.1], [0.2])
      current_activations_reshape = current_activations.reshape(current_activations.shape[0],-1)

      self.derivatives[i] = np.dot(current_activations_reshape, delta_reshaped)

      error = np.dot(delta, self.weights[i].T)  #calcular el error de la capa anterior ((y - a_[i+1]) * s'(h_[i+1]) * W_i)

      if verbose:
        print("Derivatives for W{}: {}".format(i,self.derivatives[i]))

    return error

  def train(self, inputs, targets, epochs, learning_rate):

    for i in range(epochs):
      sum_error = 0
      for (input,target) in zip(inputs,targets):
        # forward propagation
        output = self.forward_propagate(input)
        # calcula error
        error = target - output
        # backpropagation
        self.back_propagate(error)
        # actualiza pesos (gradiente decendiente)
        self.gradient_descent(learning_rate)
        #report error para esta epoca (se acumula)
        sum_error += self._mse(target, output)

      print("Error: {} at epoch {}".format(sum_error/len(inputs), i))

  def gradient_descent(self, learning_rate):
    for i in range(len(self.weights)):
      weights = self.weights[i]
      derivatives = self.derivatives[i]
      weights += derivatives * learning_rate

  def _sigmoid_derivative(self,x):
    """
    Calcula la derivada de la funcion sigmoid
    Args:
      x (float): value to be processed
    Returns:
      y (float): output
    """
    return x * (1.0 - x)

  def _mse(self, target, output):
    """
    Calcula el error cuadratico medio
    Args:
      target (ndarray): el target de la red
      output (ndarray): la salida de la red
    Returns:
      error
    """
    return np.average((target - output) ** 2)

  def _sigmoid(self, x):
    """
    Sigmoid activation function
    Args:
      x (float): value to be processed
    Returns:
      y (float): output
    """
    return 1 / (1 + np.exp(-x))



In [38]:
if __name__ == "__main__":

  # create a dataset to train a network for the sum operation
  inputs = np.array([[random()/2 for _ in range(2)] for _ in range(1000)]) # array([[0.1, 0.2], [0.3,0.4]])
  targets = np.array([[i[0] + i[1]] for i in inputs]) # array([[0.3], [0.7]])

  # crear un MLP
  mlp = MLP(2,[5],1)

  # entrenar el MLP
  mlp.train(inputs, targets, 150, 0.1)

    # crear data dummy
  input = np.array([0.3, 0.1])
  target = np.array([0.4])

  # hacer una prediccion
  output = mlp.forward_propagate(input)
  print()
  print()
  print("Our network believes that {} + {} is equal to {}".format(input[0], input[1], output[0]))

Error: 0.04863223396444173 at epoch 0
Error: 0.04342168084833459 at epoch 1
Error: 0.04293793488705722 at epoch 2
Error: 0.04232214833191854 at epoch 3
Error: 0.04152217308588721 at epoch 4
Error: 0.04047803815828766 at epoch 5
Error: 0.039124162177527004 at epoch 6
Error: 0.03739564096923335 at epoch 7
Error: 0.035240361899328415 at epoch 8
Error: 0.032637193455507904 at epoch 9
Error: 0.029616741458744045 at epoch 10
Error: 0.02627563388232458 at epoch 11
Error: 0.022772376170959358 at epoch 12
Error: 0.019298942146415938 at epoch 13
Error: 0.01603716630603756 at epoch 14
Error: 0.013120396494220346 at epoch 15
Error: 0.010616776862757096 at epoch 16
Error: 0.008535387755638073 at epoch 17
Error: 0.006844942907921382 at epoch 18
Error: 0.005493698610431743 at epoch 19
Error: 0.004424363852896708 at epoch 20
Error: 0.003582855120578137 at epoch 21
Error: 0.00292225583746748 at epoch 22
Error: 0.002403826032237494 at epoch 23
Error: 0.0019965109626671586 at epoch 24
Error: 0.0016758506